<a href="https://colab.research.google.com/github/viswavich/DLTBACS-PROJECT/blob/main/FINAL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ STEP 1: Install dependencies (Run only in Colab)
!pip install -q pandas numpy scikit-learn tensorflow shap matplotlib

In [2]:
# ✅ STEP 2: Import libraries
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

In [4]:
# ✅ STEP 3: Load datasets and trained model
df = pd.read_csv("/content/Employee_Metadata_Lookup.csv")  # Original CSV (non-encoded)
df_encoded = pd.read_csv("/content/IT_Company_1000_Employees_Encoded.csv")  # Encoded for SHAP
model = load_model("/content/dlbac_light_model.h5")  # Trained lightweight model

In [5]:
# ✅ STEP 4: Prepare encoders
label_cols = ['DEPT', 'ROLE', 'ACCESS_TIME', 'DEVICE_TRUST', 'RESOURCE_TYPE', 'SENSITIVITY_LEVEL']
encoders = {}
for col in label_cols:
    le = LabelEncoder()
    le.fit(df[col])
    encoders[col] = le

In [6]:
# ✅ STEP 5: Access Prediction Function
def get_access_decision(emp_id, resource_id):
    record = df[(df['EMP_ID'] == emp_id) & (df['RESOURCE_ID'] == resource_id)]

    if record.empty:
        return None, None, "❌ Invalid EMP_ID or RESOURCE_ID"

    print(f"🔍 Role: {record['ROLE'].values[0]}, Department: {record['DEPT'].values[0]}")

    features = [
        encoders['DEPT'].transform([record['DEPT'].values[0]])[0],
        encoders['ROLE'].transform([record['ROLE'].values[0]])[0],
        encoders['ACCESS_TIME'].transform([record['ACCESS_TIME'].values[0]])[0],
        encoders['DEVICE_TRUST'].transform([record['DEVICE_TRUST'].values[0]])[0],
        encoders['RESOURCE_TYPE'].transform([record['RESOURCE_TYPE'].values[0]])[0],
        encoders['SENSITIVITY_LEVEL'].transform([record['SENSITIVITY_LEVEL'].values[0]])[0]
    ]

    input_vector = np.array(features).reshape(1, -1)
    prediction = model.predict(input_vector)[0][0]
    decision = "✅ Access Granted" if prediction >= 0.5 else "❌ Access Denied"
    print("🔐 Access Decision:", decision)
    print("🧠 Confidence Score:", round(prediction, 4))

    return input_vector, record, prediction

In [7]:
# ✅ STEP 6: Run the Application
emp_id = 1013  # <-- Change to valid EMP_ID
resource_id = 'res404'  # <-- Change to valid RESOURCE_ID from your CSV

input_vector, record, pred = get_access_decision(emp_id, resource_id)

In [8]:
# ✅ STEP 7: SHAP Explanation (XAI)
if input_vector is not None:
    explainer = shap.Explainer(model, df_encoded.drop(['EMP_ID', 'RESOURCE_ID', 'ACTION'], axis=1).values)
    shap_values = explainer(input_vector)
    shap.plots.waterfall(shap_values[0])

In [9]:
# ✅ STEP 8: Adversarial Input Simulation (Security)
def adversarial_input(data):
    noise = np.random.normal(0, 0.1, data.shape)
    return data + noise

if input_vector is not None:
    adv_input = adversarial_input(input_vector)
    adv_pred = model.predict(adv_input)[0][0]
    adv_decision = "✅ Access Granted" if adv_pred >= 0.5 else "❌ Access Denied"
    print("⚠️ Adversarial Test:")
    print("Prediction:", round(adv_pred, 4), "|", adv_decision)

In [29]:
emp_id = 1013
resource_id = 'res974'
input_vector, record, pred = get_access_decision(emp_id, resource_id)

🔍 Role: Support, Department: R&D
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
🔐 Access Decision: ✅ Access Granted
🧠 Confidence Score: 0.8878


In [25]:
def get_access_decision(emp_id, resource_id):
    record = df[(df['EMP_ID'] == emp_id) & (df['RESOURCE_ID'] == resource_id)]

    if record.empty:
        print("❌ ERROR: Invalid EMP_ID or RESOURCE_ID. Please try again.")
        return None, None, None  # return gracefully

    print(f"🔍 Role: {record['ROLE'].values[0]}, Department: {record['DEPT'].values[0]}")

    features = [
        encoders['DEPT'].transform([record['DEPT'].values[0]])[0],
        encoders['ROLE'].transform([record['ROLE'].values[0]])[0],
        encoders['ACCESS_TIME'].transform([record['ACCESS_TIME'].values[0]])[0],
        encoders['DEVICE_TRUST'].transform([record['DEVICE_TRUST'].values[0]])[0],
        encoders['RESOURCE_TYPE'].transform([record['RESOURCE_TYPE'].values[0]])[0],
        encoders['SENSITIVITY_LEVEL'].transform([record['SENSITIVITY_LEVEL'].values[0]])[0]
    ]

    input_vector = np.array(features).reshape(1, -1)
    prediction = model.predict(input_vector)[0][0]
    decision = "✅ Access Granted" if prediction >= 0.5 else "❌ Access Denied"
    print("🔐 Access Decision:", decision)
    print("🧠 Confidence Score:", round(prediction, 4))

    return input_vector, record, prediction

In [30]:
# Example: You can change the EMP_ID and RESOURCE_ID based on your CSV
emp_id = 1013
resource_id = 'res977'

input_vector, record, pred = get_access_decision(emp_id, resource_id)

❌ ERROR: Invalid EMP_ID or RESOURCE_ID. Please try again.
